In [16]:
import cartopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
import numpy as np
from siphon.catalog import get_latest_access_url
from siphon.ncss import NCSS
import xarray as xr
import time

class HRRRCast():
    def __init__(self, lat, lon):
        self.var = [
                    'u-component_of_wind_height_above_ground',
                    'v-component_of_wind_height_above_ground',
                    'Temperature_height_above_ground',
                    'Pressure_surface',
                    'Temperature_isobaric',
                    'Pressure_reduced_to_MSL_msl',
                    'Geopotential_height_surface',
                    'Total_cloud_cover_entire_atmosphere',
                    'Temperature_height_above_ground',
                    'Total_precipitation_surface_1_Hour_Accumulation',
                    'Convective_available_potential_energy_surface',
                    'Surface_lifted_index_isobaric_layer',
                    'Wind_speed_gust_surface',
                    'Hourly_Maximum_of_Upward_Vertical_Velocity_in_the_lowest_400hPa_pressure_difference_layer_Mixed_intervals_Maximum',
                    ]
        self.north =lat+1
        self.south =lat-1
        self.east=lon+1
        self.west=lon-1
        self.latest_access_url = None
        self.nc = None
    
    def update(self):
        #'''
        latest_access_url = get_latest_access_url("http://thredds.ucar.edu/thredds/catalog/grib/NCEP/HRRR/CONUS_2p5km/catalog.xml", "NetcdfSubset")
        if self.latest_access_url == latest_access_url:
            return
        self.latest_access_url = latest_access_url
        ncss = NCSS(latest_access_url)
        query = ncss.query()
        query.accept('netcdf4')
        query.all_times()
        query.lonlat_box(north=self.north, south=self.south, east=self.east, west=self.west)
        query.variables(frozenset(self.var))
        open('temp.nc4', 'wb').write(ncss.get_data_raw(query))
        #'''
        self.nc = xr.load_dataset('temp.nc4')
        xr.set_options(keep_attrs=True)
        
        self.nc['Inverse_Surface_lifted_index'] = -self.nc['Surface_lifted_index_isobaric_layer']
        self.nc['Lapse_rate_surface_to_70kPa'] = (10000*(self.nc['Temperature_height_above_ground'][:,:,:].where(self.nc['Pressure_surface']>self.nc['isobaric1'][1])-self.nc['Temperature_isobaric'][:,1,:,:])/(self.nc['Pressure_surface']-self.nc['isobaric1'][1]))
        self.nc['Lapse_rate_surface_to_50kPa'] = (10000*(self.nc['Temperature_height_above_ground'][:,:,:].where(self.nc['Pressure_surface']>self.nc['isobaric1'][0])-self.nc['Temperature_isobaric'][:,0,:,:])/(self.nc['Pressure_surface']-self.nc['isobaric1'][0]))
        self.nc['Lapse_rate_70kPa_to_50kPa'] = (10000*(self.nc['Temperature_isobaric'][:,1,:,:].where(self.nc['Pressure_surface']>self.nc['isobaric1'][1])-self.nc['Temperature_isobaric'][:,0,:,:])/(self.nc['isobaric1'][1]-self.nc['isobaric1'][0]))
        self.nc['Wind_energy_10m'] = np.square(self.nc['u-component_of_wind_height_above_ground'][:,0])+np.square(self.nc['v-component_of_wind_height_above_ground'][:,0])
        self.nc['Wind_speed_10m'] = np.sqrt(self.nc['Wind_energy_10m'])
        self.nc['Wind_energy_80m'] = np.square(self.nc['u-component_of_wind_height_above_ground'][:,1])+np.square(self.nc['v-component_of_wind_height_above_ground'][:,1])
        self.nc['Wind_speed_80m'] = np.sqrt(self.nc['Wind_energy_80m'])
        self.nc['Glass_energy'] = \
            np.square(self.nc['u-component_of_wind_height_above_ground'][:,1]-self.nc['u-component_of_wind_height_above_ground'][:,0]) + \
            np.square(self.nc['v-component_of_wind_height_above_ground'][:,1]-self.nc['v-component_of_wind_height_above_ground'][:,0])
        self.nc['Glass'] = np.sqrt(self.nc['Glass_energy'])
        self.nc['Temperature_height_above_ground_celsius'] = self.nc['Temperature_height_above_ground']-273.15

        self.var = list(self.nc.keys())
        self.var.sort()
        
    def forecast(self, var_name = None, var_index = None, barbs = True, streamlines = False, grid_size = 1, hour = '00'):
        nc = self.nc
        var = self.var
        if (var_name is None) and (var_index is not None):
            var_name = var[var_index]
        ncvar = nc[var_name]
        ncwindu = nc['u-component_of_wind_height_above_ground']
        ncwindv = nc['v-component_of_wind_height_above_ground']
        ncvartime = ncvar.dims[0]
        ncwindtime = ncwindu.dims[0]
        ncname = ncvar.name
        nccmap = 'coolwarm'
        ncvcenter = 0
        ncvmin = None
        ncvmax = None
        nclabel = ''
        
        if ncname == 'Lapse_rate_surface_to_50kPa':
            ncvmin = 5
            ncvcenter = 10
            ncvmax = 15
            nclabel = '˚C/km'
        if ncname == 'Lapse_rate_surface_to_70kPa':
            ncvmin = 5
            ncvcenter = 10
            ncvmax = 15
            nclabel = '˚C/km'
        if ncname == 'Lapse_rate_70kPa_to_50kPa':
            ncvmin = 5
            ncvcenter = 10
            ncvmax = 15
            nclabel = '˚C/km'
        if ncname == 'Inverse_Surface_lifted_index':
            ncvmin = -5
            ncvmax = 5
        if ncname == 'Wind_speed_gust_surface':
            ncvmin = 0
            ncvcenter = 5
            ncvmax = 10
            nclabel = 'm/s'
        if ncname == 'Wind_energy_10m':
            ncvmin = 0
            ncvcenter = 25
            ncvmax = 100
            nclabel = '$kg \cdot m^{2} s^{-2}$'
        if ncname == 'Wind_speed_10m':
            ncvmin = 0
            ncvcenter = 5
            ncvmax = 10
            nclabel = 'm/s'
        if ncname == 'Wind_speed_80m':
            ncvmin = 0
            ncvcenter = 5
            ncvmax = 10
            nclabel = 'm/s'
        if ncname == 'Hourly_Maximum_of_Upward_Vertical_Velocity_in_the_lowest_400hPa_pressure_difference_layer_Mixed_intervals_Maximum':
            ncname = 'Hourly_Maximum_of_Upward_Vertical_Velocity'
            ncvmin = 0
            ncvcenter = 2
            ncvmax = 4
            nclabel = 'm/s'
        if ncname == 'Total_cloud_cover_entire_atmosphere':
            ncvmin = 0
            ncvcenter = 50
            ncvmax = 100
            nclabel = 'percent'
        if ncname == 'Convective_available_potential_energy_surface':
            ncvmax = 500
        if ncname == 'Temperature_height_above_ground_celsius':
            ncvmin = 0
            ncvcenter = 15
            ncvmax = 30
            nclabel = '˚C'
        if ncname == 'Total_precipitation_surface_1_Hour_Accumulation':
            ncvmax = 10
            nclabel = 'milimeter'
        if ncname == 'Geopotential_height_surface':
            nccmap = 'terrain'
            ncvmin = 0
            ncvcenter = 2000
            nvvmax = 4000
            nclabel = 'meters above sea level'
        if ncname == 'Glass':
            ncvmin = -1
            ncvcenter = 2
            ncvmax = 5
            nclabel = 'm/s'
            
            
        # some variables have different times arrays
        
        if len(str(hour)) == 1:
            hour = '0'+str(hour)
        
        times_found = 0
        
        ncvartime_index = 0
        for time in nc[ncvartime].data:
            if 'T'+str(hour) in np.datetime_as_string(time):
                times_found += 1
                break
            ncvartime_index += 1

        ncwindtime_index = 0
        for time in nc[ncwindtime].data:
            if 'T'+str(hour) in np.datetime_as_string(time):
                times_found += 1
                break
            ncwindtime_index += 1
        
        if times_found != 2:
            print('Requested time not in dataset')
            return
        
        #km to meters conversion
        x = ncvar.x.data*1000.
        y = ncvar.y.data*1000.
                 
        grid = nc[ncvar.grid_mapping]
        crs = ccrs.LambertConformal(
            central_longitude=grid.longitude_of_central_meridian, 
            central_latitude=grid.latitude_of_projection_origin, 
            standard_parallels=(grid.latitude_of_projection_origin,grid.standard_parallel), 
            globe=ccrs.Globe(ellipse='sphere', semimajor_axis=grid.earth_radius)
        )
                 
        fig = plt.figure(figsize=(10,10), dpi=200)
        ax = plt.axes(projection=ccrs.PlateCarree())
        
        plt.xlim([self.west,self.east])
        plt.ylim([self.south, self.north])
        
        mesh = ax.pcolorfast(x,y,ncvar[ncvartime_index,::].data.squeeze(), transform=crs, cmap=nccmap, norm = mcolors.TwoSlopeNorm(ncvcenter, vmin=ncvmin, vmax=ncvmax))
        
        if 'Wind' in ncname:
            if barbs: ax.barbs(x,y,ncwindu[ncwindtime_index,0,::].data.squeeze(),ncwindv[ncwindtime_index,0,::].data.squeeze(), transform=crs, length=3, linewidth=0.2)
            if streamlines: ax.streamplot(x,y,ncwindu[ncwindtime_index,0,::].data.squeeze(),ncwindv[ncwindtime_index,s0,::].data.squeeze(), transform=crs, density=5, color='grey', arrowsize=0.5, linewidth=0.5)
        
        if 'Glass' in ncname:
            if barbs: 
                ax.barbs(
                    x,y,
                    (ncwindu[ncwindtime_index,1,::].data.squeeze()-ncwindu[ncwindtime_index,0,::].data.squeeze())*10,
                    (ncwindv[ncwindtime_index,1,::].data.squeeze()-ncwindv[ncwindtime_index,0,::].data.squeeze())*10, 
                    transform=crs, length=3, linewidth=0.2
                )
            
        ax.scatter(-117.250, 32.890, facecolors='black', edgecolors='none', s=3)
        ax.scatter(-117.250, 32.890,  facecolors='none', edgecolors='black', marker='o', s=600)
        ax.text(-117.250+.04, 32.890-.05, ' Torrey Pines', size=9)
        ax.scatter(-116.852, 32.873, facecolors='black', edgecolors='none', s=3)
        ax.scatter(-116.852, 32.873, facecolors='none', edgecolors='black', marker='o', s=600)
        ax.text(-116.852+.04, 32.873+.05, ' Blossom', size=9)
        ax.scatter(-117.121, 32.988, facecolors='black', edgecolors='none', s=3)
        ax.scatter(-117.121, 32.988, facecolors='none', edgecolors='black', marker='o', s=600)
        ax.text(-117.121+.04, 32.988+.05, ' Little Black', size=9)
        ax.scatter(-116.953, 33.335, facecolors='black', edgecolors='none', s=3)
        ax.scatter(-116.953, 33.335, facecolors='none', edgecolors='black', marker='o', s=600)
        ax.text(-116.953+.04, 33.335+.05, ' Palomar', size=9)
        ax.scatter(-116.476, 32.774, facecolors='black', edgecolors='none', s=3)
        ax.scatter(-116.476, 32.774, facecolors='none', edgecolors='black', marker='o', s=600)
        ax.text(-116.476+.04, 32.774-.05, ' Horse', size=9)
        ax.scatter(-116.418, 32.880, facecolors='black', edgecolors='none', s=3)
        ax.scatter(-116.418, 32.880, facecolors='none', edgecolors='black', marker='o', s=600)
        ax.text(-116.418+.04, 32.880+.05, ' Laguna', size=9)
        ax.scatter(-117.370, 33.629, facecolors='black', edgecolors='none', s=3)
        ax.scatter(-117.370, 33.629, facecolors='none', edgecolors='black', marker='o', s=600)
        ax.text(-117.370+.04, 33.629+.05, ' Elsinore', size=9)
        ax.scatter(-116.809, 33.157, facecolors='black', edgecolors='none', s=3)
        ax.scatter(-116.809, 33.157, facecolors='none', edgecolors='black', marker='o', s=600)
        ax.text(-116.809+.04, 33.157+.05, ' Big Black', size=9)
        ax.scatter(-116.893, 32.573, facecolors='black', edgecolors='none', s=3)
        ax.scatter(-116.893, 32.573, facecolors='none', edgecolors='black', marker='o', s=600)
        ax.text(-116.893+.04, 32.573+.05, ' Otay', size=9)
        ax.scatter(-116.957, 33.821, facecolors='black', edgecolors='none', s=3)
        ax.scatter(-116.957, 33.821, facecolors='none', edgecolors='black', marker='o', s=600)
        ax.text(-116.957+.04, 33.821+.05, ' Soboba', size=9)       
        
        # ax.add_feature(cartopy.feature.NaturalEarthFeature('cultural', 'roads', '10m'), edgecolor='black', facecolor='none')

        
        gl = ax.gridlines(draw_labels=True, color = 'black', linewidth=0.5)
        gl.xlocator = mticker.FixedLocator(np.arange(self.west,self.east+grid_size,grid_size))
        gl.ylocator = mticker.FixedLocator(np.arange(self.south,self.north+grid_size,grid_size))
        gl.top_labels = False
        gl.right_labels = False
        
        plt.colorbar(mesh, label=nclabel, shrink=0.8)
        plt.title(str(nc[ncvartime].data[ncvartime_index])[:-16] + ' ' + ncname);
        plt.show()

In [17]:
hr = HRRRCast(33.2,-117)

In [18]:
hr.update()

In [19]:
hr.nc

In [20]:
hr.var

In [22]:
# hr.update()
hr.forecast(var_name = 'Geopotential_height_surface', hour = 18)
for x in [12,13,14,15,16,17,18,19,20,21,22,23,0,1,2,3,4]:
    hr.forecast(var_name = 'Wind_speed_10m', hour = x)
    hr.forecast(var_name = 'Wind_speed_gust_surface', hour = x)
    hr.forecast(var_name = 'Glass', hour = x)
    # hr.forecast(var_name = 'Wind_speed_80m', hour = x)
    # hr.forecast(var_name = 'Wind_energy_10m', hour = x)
    # hr.forecast(var_name = 'Temperature_height_above_ground_celsius', hour = x)
    hr.forecast(var_name = 'Hourly_Maximum_of_Upward_Vertical_Velocity_in_the_lowest_400hPa_pressure_difference_layer_Mixed_intervals_Maximum', hour = x)
    # hr.forecast(var_name = 'Lapse_rate_surface_to_70kPa', hour = x)
    # hr.forecast(var_name = 'Lapse_rate_70kPa_to_50kPa', hour = x)
    # hr.forecast(var_name = 'Total_precipitation_surface_1_Hour_Accumulation', hour = x)
    hr.forecast(var_name = 'Total_cloud_cover_entire_atmosphere', hour = x)

